In [ ]:
import numpy as np
import pandas as pd
import os

# Image processing libraries
from PIL import Image
import matplotlib.pyplot as plt

# Deep Learning Framework: TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Conv2D, Conv2DTranspose, Flatten, LeakyReLU, Dropout, BatchNormalization, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.data import AUTOTUNE

print("--- Required Libraries Imported Successfully ---")
print(f"TensorFlow Version: {tf.__version__}")

2025-12-23 14:32:59.833264: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766500380.058514      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766500380.121672      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766500380.640806      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766500380.640848      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766500380.640851      55 computation_placer.cc:177] computation placer alr

--- Required Libraries Imported Successfully ---
TensorFlow Version: 2.19.0


In [ ]:
import os
import tensorflow as tf

# 1. MirroredStrategy (Donon GPUs ke liye)
strategy = tf.distribute.MirroredStrategy()
num_gpus = strategy.num_replicas_in_sync

# 2. Path Setup (Ultra-fast globbing)
base_path = "/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba"
if not os.path.exists(base_path):
    base_path = "/kaggle/input/celeba-dataset/img_align_celeba"

# tf.io.gfile.glob python ke os.walk se kahin zyada tez hai
img_roots = tf.io.gfile.glob(base_path + "/*.jpg")
print(f"Total images found: {len(img_roots)}")

# 3. Parameters
AUTOTUNE = tf.data.AUTOTUNE 
BATCH_SIZE_PER_REPLICA = 64 
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * num_gpus

# 4. Preprocessing Function
def tf_load_and_preprocess(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [128, 128])
    # float16 use karne se memory aur speed donon behtar hoti hain
    img = tf.cast(img, tf.float16) 
    img = (img / 127.5) - 1.0
    return img

# 5. Parallelism & Multithreading Pipeline
dataset = tf.data.Dataset.from_tensor_slices(img_roots)

# --- Multithreading Step 1: Parallel Mapping ---
# num_parallel_calls=AUTOTUNE background mein threads manage karta hai
dataset = dataset.map(tf_load_and_preprocess, num_parallel_calls=AUTOTUNE)

# --- Optimization Step 2: Caching ---
# Agar RAM kam paray to .cache() ko skip kar sakte hain
dataset = dataset.cache() 

# --- Optimization Step 3: Shuffling & Batching ---
dataset = dataset.shuffle(buffer_size=2048)
dataset = dataset.batch(GLOBAL_BATCH_SIZE, drop_remainder=True)

# --- Multithreading Step 4: Prefetching ---
# Ye training aur data loading ko overlap karta hai
dataset = dataset.prefetch(buffer_size=AUTOTUNE)

# --- Multi-GPU Sharding ---
dist_dataset = strategy.experimental_distribute_dataset(dataset)

print(f"✅ Multithreaded Pipeline Ready! Global Batch: {GLOBAL_BATCH_SIZE}")


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


I0000 00:00:1766500394.416124      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1766500394.420141      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Total images found: 202599
✅ Multithreaded Pipeline Ready! Global Batch: 128


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Input, Dense, Reshape, Conv2D, Conv2DTranspose, LeakyReLU, Flatten, Concatenate, UpSampling2D
from tensorflow.keras.models import Model

# 1. Pixelwise Normalization (Generator ke liye)
# Yeh layer training ko explode hone se bachaati hai
class PixelNormalization(Layer):
    def __init__(self, epsilon=1e-7, **kwargs): # Epsilon thoda barha diya
        super(PixelNormalization, self).__init__(**kwargs)
        self.epsilon = epsilon

    def call(self, inputs):
        # inputs ko float32 mein cast karna taake math stable rahe
        inputs = tf.cast(inputs, tf.float32)
        return inputs * tf.math.rsqrt(tf.reduce_mean(tf.square(inputs), axis=-1, keepdims=True) + self.epsilon)

class MiniBatchStd(Layer):
    def __init__(self, **kwargs):
        super(MiniBatchStd, self).__init__(**kwargs)

    def call(self, inputs):
        inputs = tf.cast(inputs, tf.float32)
        group_std = tf.math.reduce_std(inputs, axis=0, keepdims=True)
        avg_std = tf.reduce_mean(group_std, keepdims=True)
        shape = tf.shape(inputs)
        avg_std = tf.tile(avg_std, [shape[0], shape[1], shape[2], 1])
        return tf.concat([inputs, avg_std], axis=-1)

print("✅ ProGAN Custom Layers defined.")

✅ ProGAN Custom Layers defined.


In [ ]:
noise_dim = 128

class ProGANGenerator(tf.keras.Model):
    def __init__(self, noise_dim, initial_dim=4, **kwargs):
        super(ProGANGenerator, self).__init__(**kwargs)
        self.noise_dim = noise_dim
        
        # We use a standard Sequential for the main blocks to ensure weight sync
        self.model_layers = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(noise_dim,)),
            tf.keras.layers.Dense(initial_dim * initial_dim * 512, use_bias=False),
            tf.keras.layers.Reshape((initial_dim, initial_dim, 512)),
            tf.keras.layers.LeakyReLU(0.2),
            PixelNormalization(),
            
            # 4x4 -> 8x8
            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(512, (3, 3), padding="same", use_bias=False),
            tf.keras.layers.LeakyReLU(0.2),
            PixelNormalization(),
            
            # 8x8 -> 16x16
            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(256, (3, 3), padding="same", use_bias=False),
            tf.keras.layers.LeakyReLU(0.2),
            PixelNormalization(),
            
            # 16x16 -> 32x32
            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(128, (3, 3), padding="same", use_bias=False),
            tf.keras.layers.LeakyReLU(0.2),
            PixelNormalization(),
            
            # 32x32 -> 64x64
            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(64, (3, 3), padding="same", use_bias=False),
            tf.keras.layers.LeakyReLU(0.2),
            PixelNormalization(),
            
            # 64x64 -> 128x128
            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(32, (3, 3), padding="same", use_bias=False),
            tf.keras.layers.LeakyReLU(0.2),
            PixelNormalization(),
            
            # Final RGB Layer
            tf.keras.layers.Conv2D(3, (1, 1), padding="same", activation="tanh")
        ])

    def call(self, inputs, training=False):
        return self.model_layers(inputs, training=training)

# --- CRITICAL RE-INITIALIZATION ---
with strategy.scope():
    print("💎 Re-instantiating Generator for Dual T4 Sync...")
    generator = ProGANGenerator(noise_dim)

    _ = generator(tf.random.normal((GLOBAL_BATCH_SIZE, noise_dim)))

print(f"✅ Weights initialized. Trainable variables: {len(generator.trainable_variables)}")


💎 Re-instantiating Generator for Dual T4 Sync...


I0000 00:00:1766500490.140004      55 cuda_dnn.cc:529] Loaded cuDNN version 91002


✅ Weights initialized. Trainable variables: 8


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

class ProGANDiscriminator(tf.keras.Model):
    def __init__(self, **kwargs):
        super(ProGANDiscriminator, self).__init__(**kwargs)
        
        # ProGAN ki depth ke mutabiq filters
        self.filter_list = [32, 64, 128, 256, 512]
        self.conv_blocks = []
        for f in self.filter_list:
            self.conv_blocks.append(layers.Conv2D(f, (3, 3), padding="same"))
            
        self.pool = layers.AveragePooling2D(pool_size=(2, 2))
        self.leaky = layers.LeakyReLU(0.2)
        
        # Custom layer jo humne Cell 3 mein define ki thi
        self.minibatch_std = MiniBatchStd()
        
        self.conv_final = layers.Conv2D(512, (3, 3), padding="same")
        self.flatten = layers.Flatten()
        self.dense_decision = layers.Dense(1)

    def call(self, img):
        x = img
        for conv in self.conv_blocks:
            x = conv(x)
            x = self.leaky(x)
            x = self.pool(x)
        
        # Multi-GPU par ye local replica ke stats use karega
        x = self.minibatch_std(x)
        x = self.conv_final(x)
        x = self.leaky(x)
        x = self.flatten(x)
        return self.dense_decision(x)

# --- Multi-GPU Build and Summary ---
with strategy.scope():
    print("🚀 Initializing Discriminator on both T4 GPUs...")
    discriminator = ProGANDiscriminator()

    dummy_img = tf.random.normal((2, 128, 128, 3))
    _ = discriminator(dummy_img)

# Summary check
discriminator.summary()
print("✅ Discriminator is now mirrored across 2 GPUs.")


🚀 Initializing Discriminator on both T4 GPUs...


Model: "pro_gan_discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (2, 128, 128, 32)      │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (2, 64, 64, 64)        │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (2, 32, 32, 128)       │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (2, 16, 16, 256)       │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (2, 8, 8, 512)         │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ ?                      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_6 (LeakyReLU)       │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mini_batch_std (MiniBatchStd)   │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (2, 4, 4, 512)         │     2,364,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (2, 8192)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (2, 1)                 │         8,193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,941,185 (15.03 MB)

 Trainable params: 3,941,185 (15.03 MB)

 Non-trainable params: 0 (0.00 B)

✅ Discriminator is now mirrored across 2 GPUs.


In [ ]:
import tensorflow as tf
from tensorflow.keras import mixed_precision

# 1. Global Policy for T4 Tensor Cores (2025 Standard)
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
print(f"✅ Global policy: {mixed_precision.global_policy()}")

# 1e-4 ProGAN ke liye sab se stable hai
generator_LR = 1e-4
discriminator_LR = 1e-4

with strategy.scope():
    print("🚀 Initializing Losses and Optimizers inside Strategy Scope...")
    
    # 3. Optimizers with Built-in Gradient Clipping
    # global_clipnorm=1.0 is the secret to preventing "Loss: NaN"
    generator_optimizer = tf.keras.optimizers.Adam(
        learning_rate=generator_LR, 
        beta_1=0.0, 
        beta_2=0.9, 
        global_clipnorm=1.0
    )
    generator_optimizer = mixed_precision.LossScaleOptimizer(generator_optimizer)

    discriminator_optimizer = tf.keras.optimizers.Adam(
        learning_rate=discriminator_LR, 
        beta_1=0.0, 
        beta_2=0.9, 
        global_clipnorm=1.0
    )
    discriminator_optimizer = mixed_precision.LossScaleOptimizer(discriminator_optimizer)

    def initialize_optimizer_slots():
        # Force Adam to create momentum slots on both GPUs to avoid Placeholder errors
        gen_vars = generator.trainable_variables
        gen_zero_grads = [tf.zeros_like(v) for v in gen_vars]
        generator_optimizer.apply_gradients(zip(gen_zero_grads, gen_vars))
        
        disc_vars = discriminator.trainable_variables
        disc_zero_grads = [tf.zeros_like(v) for v in disc_vars]
        discriminator_optimizer.apply_gradients(zip(disc_zero_grads, disc_vars))

    print("⚡ Syncing optimizer slots across T4-0 and T4-1...")
    strategy.run(initialize_optimizer_slots)

print("✅ Stable Optimizers mirrored and ready for 128x128 Training.")


✅ Global policy: <DTypePolicy "mixed_float16">
🚀 Initializing Losses and Optimizers inside Strategy Scope...
⚡ Syncing optimizer slots across T4-0 and T4-1...
✅ Stable Optimizers mirrored and ready for 128x128 Training.


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

# --- STEP 1: Define the function ---
def build_vgg_extractor():

    vgg = VGG16(include_top=False, weights="imagenet", input_shape=(128, 128, 3))
    vgg.trainable = False
    
    # Extracting a specific layer for Perceptual Loss
    # Block3_conv3 is commonly used for high-level feature matching
    output_layer = vgg.get_layer("block3_conv3").output
    
    model = Model(inputs=vgg.input, outputs=output_layer, name="VGG_Feature_Extractor")
    return model


with strategy.scope():
    print("📥 Loading and Mirroring VGG16 on both GPUs...")
    vgg_feature_extractor = build_vgg_extractor()
    
    dummy_input = tf.random.normal((2, 128, 128, 3))
    _ = vgg_feature_extractor(dummy_input)

# --- STEP 3: Summary Check ---
vgg_feature_extractor.summary()
print("✅ VGG Feature Extractor is now distributed and ready for Loss Calculation.")


📥 Loading and Mirroring VGG16 on both GPUs...
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "VGG_Feature_Extractor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,735,488 (6.62 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,735,488 (6.62 MB)

✅ VGG Feature Extractor is now distributed and ready for Loss Calculation.


In [ ]:
import os
import matplotlib.pyplot as plt

# --- 1. Directory Setup ---
CHECKPOINT_DIR = './training_checkpoints_128'
if not os.path.exists(CHECKPOINT_DIR):
    os.makedirs(CHECKPOINT_DIR)

# --- 2. Fixed Seed for Testing ---
num_examples_to_generate = 16
seed_noise = tf.random.normal([num_examples_to_generate, noise_dim])

# --- 3. Optimized Image Saving Function ---
def generate_and_save_images(model, test_noise, epoch, name):
    predictions = model(test_noise, training=False)
    predictions = tf.cast(predictions, tf.float32).numpy()

    fig = plt.figure(figsize=(10, 10))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        # Tanh (-1 to 1) se image format (0 to 1) mein conversion
        img = (predictions[i] + 1.0) / 2.0
        # Clipping taake floating point errors ki wajah se pixels out of range na hon
        img = np.clip(img, 0.0, 1.0)
        plt.imshow(img)     
        plt.axis('off')

    save_path = os.path.join(CHECKPOINT_DIR, f'image_at_{name}_epoch_{epoch:04d}.png')
    plt.savefig(save_path)
    plt.close(fig) # RAM management ke liye zaroori hai

print("✅ Image saving function optimized for Multi-GPU environment.")

# --- 4. Distributed Checkpoint Manager ---
with strategy.scope():
    checkpoint = tf.train.Checkpoint(
        generator_optimizer=generator_optimizer,
        discriminator_optimizer=discriminator_optimizer,
        generator=generator,
        discriminator=discriminator
    )

    checkpoint_manager = tf.train.CheckpointManager(
        checkpoint, CHECKPOINT_DIR, max_to_keep=3
    )

# Agar purana checkpoint mojood hai toh restore karein
if checkpoint_manager.latest_checkpoint:
    checkpoint.restore(checkpoint_manager.latest_checkpoint)
    print(f"✅ Restored from {checkpoint_manager.latest_checkpoint}")
else:
    print("🆕 Starting training from scratch.")

print("✅ Visualization and Checkpoint system ready for Dual T4 GPUs.")


✅ Image saving function optimized for Multi-GPU environment.
🆕 Starting training from scratch.
✅ Visualization and Checkpoint system ready for Dual T4 GPUs.


In [ ]:
import time

# --- 1. Distributed Train Step ---
@tf.function
def distributed_pre_train_step(dataset_inputs):
    # strategy.run distributes the logic across GPU-0 and GPU-1
    per_replica_losses = strategy.run(train_step, args=(dataset_inputs,))
    
    # Aggregate losses from both GPUs
    return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)

def train_step(images):
    # Use global static batch size per GPU (64)
    # This avoids dynamic 'Placeholder' errors in the graph
    batch_size = BATCH_SIZE_PER_REPLICA 

    with tf.GradientTape() as tape:
        # Generate Noise
        noise = tf.random.normal(shape=(batch_size, noise_dim))
        
        # Forward Pass
        generated_images = generator(noise, training=True)
        
        # Sync types to float32 for high-precision MAE loss
        images_f32 = tf.cast(images, tf.float32)
        generated_f32 = tf.cast(generated_images, tf.float32)
        
        # Mean Absolute Error Loss
        loss = tf.reduce_mean(tf.abs(images_f32 - generated_f32))
        
        # Scale loss for mixed_float16 stability
        scaled_loss = generator_optimizer.scale_loss(loss)

    # Gradient Calculation
    trainable_vars = generator.trainable_variables
    scaled_gradients = tape.gradient(scaled_loss, trainable_vars)

    generator_optimizer.apply_gradients(zip(scaled_gradients, trainable_vars))
    
    return loss

# --- 2. Training Loop Execution ---
EPOCHS = 5
STEPS_PER_EPOCH = 500

print(f"🚀 Starting Stabilized Dual-T4 Training...")
# Re-distribute dataset to ensure clean sharding
dist_dataset = strategy.experimental_distribute_dataset(dataset)

for epoch in range(EPOCHS):
    start_time = time.time()
    total_loss = 0.0
    step_count = 0
    
    for images in dist_dataset:
        if step_count >= STEPS_PER_EPOCH:
            break
            
        try:
            # Sync step across both GPUs
            step_loss = distributed_pre_train_step(images)
            total_loss += step_loss
            step_count += 1
            
            if step_count % 100 == 0:
                print(f"  [Epoch {epoch+1}] Batch {step_count}/{STEPS_PER_EPOCH} | Loss: {step_loss:.4f}")
                
        except Exception as e:
            print(f"❌ Critical Error: {e}")
            raise e 

    # Progress Summary
    avg_loss = total_loss / step_count
    print(f"✅ Epoch {epoch+1} done | Avg Loss: {avg_loss:.6f} | Time: {time.time()-start_time:.2f}s")

    # Save Samples and Checkpoints
    generate_and_save_images(generator, seed_noise, epoch+1, "stableize")
    if epoch%10 == 0:
        checkpoint_manager.save()

print("🏁 Training successfully utilized both GPUs!")


🚀 Starting Stabilized Dual-T4 Training...
INFO:tensorflow:Collective all_reduce tensors: 8 all_reduces, num_devices = 2, group_size = 2, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


I0000 00:00:1766500503.536985     104 cuda_dnn.cc:529] Loaded cuDNN version 91002


  [Epoch 1] Batch 100/500 | Loss: 0.9924
  [Epoch 1] Batch 200/500 | Loss: 0.9667
  [Epoch 1] Batch 300/500 | Loss: 0.9644
  [Epoch 1] Batch 400/500 | Loss: 0.9686
  [Epoch 1] Batch 500/500 | Loss: 0.9522
✅ Epoch 1 done | Avg Loss: 0.996505 | Time: 96.89s
  [Epoch 2] Batch 100/500 | Loss: 0.9551
  [Epoch 2] Batch 200/500 | Loss: 0.9268
  [Epoch 2] Batch 300/500 | Loss: 0.9665
  [Epoch 2] Batch 400/500 | Loss: 0.9488
  [Epoch 2] Batch 500/500 | Loss: 0.9599
✅ Epoch 2 done | Avg Loss: 0.954193 | Time: 82.14s
  [Epoch 3] Batch 100/500 | Loss: 0.9391
  [Epoch 3] Batch 200/500 | Loss: 0.9330
  [Epoch 3] Batch 300/500 | Loss: 0.9333
  [Epoch 3] Batch 400/500 | Loss: 0.9523
  [Epoch 3] Batch 500/500 | Loss: 0.9411
✅ Epoch 3 done | Avg Loss: 0.950559 | Time: 84.95s
  [Epoch 4] Batch 100/500 | Loss: 0.9506
  [Epoch 4] Batch 200/500 | Loss: 0.9886
  [Epoch 4] Batch 300/500 | Loss: 0.9512
  [Epoch 4] Batch 400/500 | Loss: 0.9449
  [Epoch 4] Batch 500/500 | Loss: 0.9452
✅ Epoch 4 done | Avg Loss: 

In [ ]:
@tf.function
def compute_gradient_penalty(real_images, fake_images, batch_size):
    """
    Calculates the gradient penalty.
    This is essential for WGAN-GP stability.
    """
    # 1. Get the interpolation fraction
    alpha = tf.random.uniform([batch_size, 1, 1, 1], 0.0, 1.0)
    alpha = tf.cast(alpha, tf.float32)
    
    real_images = tf.cast(real_images, tf.float32)
    fake_images = tf.cast(fake_images, tf.float32)
    
    # 2. Create interpolated images
    interpolated = real_images + alpha * (fake_images - real_images)

    with tf.GradientTape() as gp_tape:
        gp_tape.watch(interpolated)
        # 3. Get discriminator output for interpolated images
        # Important: Use your argument name 'img'
        pred = discriminator(img=interpolated, training=True)

    # 4. Calculate gradients w.r.t. interpolated images
    grads = gp_tape.gradient(pred, [interpolated])[0]
    
    # 5. Calculate the norm of the gradients
    norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]) + 1e-12)
    gp = tf.reduce_mean((norm - 1.0) ** 2)
    
    return tf.cast(gp, tf.float32)

print("✅ Gradient Penalty function defined and optimized for Mixed Precision.")


✅ Gradient Penalty function defined and optimized for Mixed Precision.


In [ ]:
import tensorflow as tf

# --- 1. High-Precision Gradient Penalty ---
def compute_gradient_penalty(real_images, fake_images, batch_size):
    alpha = tf.random.uniform([batch_size, 1, 1, 1], 0.0, 1.0, dtype=tf.float32)
    # Sab math float32 mein karein taake NaN na aaye
    real_f32 = tf.cast(real_images, tf.float32)
    fake_f32 = tf.cast(fake_images, tf.float32)
    interpolated = real_f32 + alpha * (fake_f32 - real_f32)

    with tf.GradientTape() as gp_tape:
        gp_tape.watch(interpolated)
        pred = discriminator(interpolated, training=True)
        pred = tf.cast(pred, tf.float32)

    grads = gp_tape.gradient(pred, [interpolated])[0]
    norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]) + 1e-8)
    gp = tf.reduce_mean((norm - 1.0) ** 2)
    return tf.cast(gp, tf.float32)

# --- 2. The Core Train Step ---
@tf.function
def train_step(images):
    current_batch_size = BATCH_SIZE_PER_REPLICA 

    # --- Discriminator Training ---
    for _ in range(2):
        with tf.GradientTape() as d_tape:
            noise = tf.random.normal([current_batch_size, noise_dim])
            fake_images = generator(noise, training=True)
            
            real_logits = discriminator(images, training=True)
            fake_logits = discriminator(fake_images, training=True)
            
            w_loss = tf.cast(tf.reduce_mean(fake_logits) - tf.reduce_mean(real_logits), tf.float32)
            gp = compute_gradient_penalty(images, fake_images, current_batch_size)
            d_loss = w_loss + (10.0 * gp) 
            
            scaled_d_loss = discriminator_optimizer.scale_loss(d_loss)

        d_vars = discriminator.trainable_variables
        scaled_d_grads = d_tape.gradient(scaled_d_loss, d_vars)
        # Apply gradients (Auto-unscaling & Auto-clipping)
        discriminator_optimizer.apply_gradients(zip(scaled_d_grads, d_vars))

    # --- Generator Training ---
    with tf.GradientTape() as g_tape:
        noise = tf.random.normal([current_batch_size, noise_dim])
        gen_imgs = generator(noise, training=True)
        gen_logits = discriminator(gen_imgs, training=True)
        
        g_adv_loss = tf.cast(-tf.reduce_mean(gen_logits), tf.float32)
        
        # Perceptual Loss
        real_vgg = (tf.cast(images, tf.float32) + 1.0) * 127.5
        fake_vgg = (tf.cast(gen_imgs, tf.float32) + 1.0) * 127.5
        perc_loss = tf.reduce_mean(tf.abs(vgg_feature_extractor(real_vgg) - vgg_feature_extractor(fake_vgg)))
        
        total_g_loss = g_adv_loss + (0.01 * tf.cast(perc_loss, tf.float32))
        scaled_g_loss = generator_optimizer.scale_loss(total_g_loss)

    g_vars = generator.trainable_variables
    scaled_g_grads = g_tape.gradient(scaled_g_loss, g_vars)
    generator_optimizer.apply_gradients(zip(scaled_g_grads, g_vars))

    return d_loss, total_g_loss

# --- 3. Distributed Wrapper ---
@tf.function
def distributed_train_step(dist_inputs):
    per_replica_d_losses, per_replica_g_losses = strategy.run(train_step, args=(dist_inputs,))
    # Reduce across both T4 GPUs
    mean_d = strategy.reduce(tf.distribute.ReduceOp.MEAN, per_replica_d_losses, axis=None)
    mean_g = strategy.reduce(tf.distribute.ReduceOp.MEAN, per_replica_g_losses, axis=None)
    return mean_d, mean_g

print("✅ Distributed Pipeline Refreshed.")


✅ Distributed Pipeline Refreshed.


In [ ]:
import time
import matplotlib.pyplot as plt

def train(epochs):
    print(f"🚀 Starting Main Training for 128x128 resolution on {strategy.num_replicas_in_sync} GPUs...")
    
    # Dataset ko distribute karna sab se pehle lazmi hai
    # Yeh ensures karta hai ke images dono GPUs mein barabar taqseem hon
    dist_dataset = strategy.experimental_distribute_dataset(dataset)

    for epoch in range(epochs):
        start = time.time()
        D_loss_total = 0.0
        G_loss_total = 0.0
        batch_count = 0

        # Distributed dataset par iterate karna
        for image_batch in dist_dataset:
            d_loss_replica, g_loss_replica = distributed_train_step(image_batch)
            
            D_loss_total += d_loss_replica
            G_loss_total += g_loss_replica
            batch_count += 1
        
        # Average losses nikalna
        avg_d = D_loss_total / batch_count
        avg_g = G_loss_total / batch_count
        
        print(f'✅ Epoch {epoch+1:03d} finished in {time.time()-start:.2f}s')
        print(f'   📊 Loss D: {avg_d:.4f}, Loss G: {avg_g:.4f}')

        # Har epoch ke baad samples generate karna
        generate_and_save_images(generator, seed_noise, epoch + 1, "train")

        # Har 5 epochs baad checkpoint save karna
        if (epoch + 1) % 5 == 0:
            checkpoint_manager.save()
            print(f"💾 Checkpoint saved at epoch {epoch+1}")


EPOCHS = 50

train(EPOCHS)


🚀 Starting Main Training for 128x128 resolution on 2 GPUs...
INFO:tensorflow:Collective all_reduce tensors: 14 all_reduces, num_devices = 2, group_size = 2, implementation = CommunicationImplementation.NCCL, num_packs = 1


INFO:tensorflow:Collective all_reduce tensors: 14 all_reduces, num_devices = 2, group_size = 2, implementation = CommunicationImplementation.NCCL, num_packs = 1


INFO:tensorflow:Collective all_reduce tensors: 14 all_reduces, num_devices = 2, group_size = 2, implementation = CommunicationImplementation.NCCL, num_packs = 1


INFO:tensorflow:Collective all_reduce tensors: 14 all_reduces, num_devices = 2, group_size = 2, implementation = CommunicationImplementation.NCCL, num_packs = 1
/usr/local/lib/python3.12/dist-packages/tensorflow/python/framework/tensor_util.py:578: RuntimeWarning: overflow encountered in cast
  nparray = values.astype(dtype.as_numpy_dtype)


INFO:tensorflow:Collective all_reduce tensors: 8 all_reduces, num_devices = 2, group_size = 2, implementation = CommunicationImplementation.NCCL, num_packs = 1


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


✅ Epoch 001 finished in 1257.25s
   📊 Loss D: -12.9161, Loss G: 25.5992
✅ Epoch 002 finished in 1241.13s
   📊 Loss D: -8.4022, Loss G: 20.0392
✅ Epoch 003 finished in 1235.51s
   📊 Loss D: -7.1014, Loss G: 19.0515
✅ Epoch 004 finished in 1252.54s
   📊 Loss D: -6.5513, Loss G: 22.4038
✅ Epoch 005 finished in 1237.85s
   📊 Loss D: -5.8222, Loss G: 27.5019
💾 Checkpoint saved at epoch 5
✅ Epoch 006 finished in 1235.69s
   📊 Loss D: -5.2298, Loss G: 31.9973
✅ Epoch 007 finished in 1252.79s
   📊 Loss D: -4.8970, Loss G: 30.8204
✅ Epoch 008 finished in 1236.88s
   📊 Loss D: -4.7140, Loss G: 27.0026
✅ Epoch 009 finished in 1237.67s
   📊 Loss D: -4.5309, Loss G: 23.7241
✅ Epoch 010 finished in 1255.26s
   📊 Loss D: -4.4308, Loss G: 25.3408
💾 Checkpoint saved at epoch 10
✅ Epoch 011 finished in 1241.51s
   📊 Loss D: -4.3019, Loss G: 24.8706
✅ Epoch 012 finished in 1242.38s
   📊 Loss D: -4.1842, Loss G: 26.9988
✅ Epoch 013 finished in 1256.27s
   📊 Loss D: -4.0797, Loss G: 27.2017
✅ Epoch 014 fin

KeyboardInterrupt: 

In [ ]:
import pickle

weights = generator.model_layers.get_weights()

with open('generator_weights.pkl', 'wb') as f:
    pickle.dump(weights, f)